In [1]:
import pandas as pd
import numpy as np

In [7]:
import lightgbm
import xgboost

In [6]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')

df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [8]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 9),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}

In [9]:
df.describe()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5.250000,5.250001,5.250004,5.249997,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731
std,2.742548,2.742549,2.742549,2.742556,0.752160,0.433035,0.433035,0.433035,0.274256,0.274255,0.274255,0.274255,0.036919
min,0.500793,0.500141,0.500788,0.500473,1.582590,-1.999891,-1.999945,-1.999926,0.050009,0.050053,0.050054,0.050028,-0.080760
25%,2.874892,2.875140,2.875522,2.874950,3.218300,-1.624901,-1.625025,-1.624960,0.287521,0.287552,0.287514,0.287494,-0.015557
50%,5.250004,5.249981,5.249979,5.249734,3.751025,-1.249966,-1.249974,-1.250007,0.525009,0.525003,0.525015,0.525002,0.017142
75%,7.624690,7.624893,7.624948,7.624838,4.282420,-0.874977,-0.875043,-0.875065,0.762435,0.762490,0.762440,0.762433,0.044878
max,9.999469,9.999837,9.999450,9.999443,5.864418,-0.500108,-0.500072,-0.500025,0.999937,0.999944,0.999982,0.999930,0.109403


In [10]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [12]:
df = df.drop(['stab'], axis=1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [13]:
df.head(2)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable


In [15]:
X = df.iloc[:, :-1].values
Y = df['stabf']

In [30]:
df.stabf.value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [69]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(Y)


In [70]:
(unique, counts) = np.unique(y, return_counts=True)

frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   0 3620]
 [   1 6380]]


In [71]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [72]:
from sklearn.preprocessing import StandardScaler

In [73]:
# y_train = y_train.reshape(-1,1)
# y_test = y_test.reshape(-1,1)

In [74]:
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
# Y_train = sc.fit_transform(y_train)
X_test = sc.fit_transform(x_test)

In [103]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators= 100,random_state = 1)
  
# fit the classifier with x and y data 
rf.fit(X_train, y_train)   
rf_pred = rf.predict(X_test)

In [104]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, rf_pred)

print(acc)

0.928


In [47]:
y_train.shape

(8000, 1)

In [75]:
y_train.dtype

dtype('int32')

In [77]:
etc = ExtraTreesClassifier(n_estimators = 5, criterion ='entropy', max_features = 2, random_state=1) 
  
# Training the model 
etc.fit(X_train, y_train) 
  
# Computing the importance of each feature 
feature_importance = etc.feature_importances_ 

etc_pred = etc.predict(X_test)

In [95]:
etc_acc = accuracy_score(y_test, etc_pred)
print(etc_acc)

0.8145


In [78]:
n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None] 

hyperparameter_grid = {'n_estimators': n_estimators,

                       'min_samples_leaf': min_samples_leaf,

                       'min_samples_split': min_samples_split,

                       'max_features': max_features}

etc_cv = RandomizedSearchCV(etc, hyperparameter_grid, cv=5)

In [89]:
etc2 = ExtraTreesClassifier()
etc_cv2 = RandomizedSearchCV(etc2, hyperparameter_grid, cv=5, n_iter=10, 
                             scoring = 'accuracy', n_jobs = -1, verbose = 1, random_state = 1)

In [93]:
etc_cv2.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   57.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.1min finished


RandomizedSearchCV(cv=5, estimator=ExtraTreesClassifier(), n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 3, 5, 7, 9],
                                        'n_estimators': [50, 100, 300, 500,
                                                         1000]},
                   random_state=1, scoring='accuracy', verbose=1)

In [79]:
# tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)

# # Fit it to the data
etc_cv.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=ExtraTreesClassifier(criterion='entropy',
                                                  max_features=2,
                                                  n_estimators=5,
                                                  random_state=1),
                   param_distributions={'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 3, 5, 7, 9],
                                        'n_estimators': [50, 100, 300, 500,
                                                         1000]})

In [94]:
# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(etc_cv2.best_params_))
print("Best score is {}".format(etc_cv2.best_score_))

Tuned Decision Tree Parameters: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': None}
Best score is 0.9248750000000001


In [97]:
optimal_etc = ExtraTreesClassifier(n_estimators= 1000, min_samples_split= 2, min_samples_leaf= 8,
                                   max_features= None, random_state=1)

# Training the model 
optimal_etc.fit(X_train, y_train) 
  
# Computing the importance of each feature 
feature_importance = optimal_etc.feature_importances_ 

optimal_etc_pred = optimal_etc.predict(X_test)

In [99]:
optimal_acc = accuracy_score(y_test,optimal_etc_pred )
print(optimal_acc)

0.9285


In [98]:
# Computing the importance of each feature 
feature_importance2 = optimal_etc.feature_importances_
feature_importance2

array([0.13723975, 0.1405075 , 0.13468029, 0.13541676, 0.00368342,
       0.00533686, 0.00542927, 0.00496249, 0.10256244, 0.10757765,
       0.11306268, 0.10954089])

In [102]:
df.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stabf'],
      dtype='object')

In [87]:
xgb = xgboost.XGBClassifier(learning_rate=0.1, n_estimators=140, max_depth=5,
                        min_child_weight=3, gamma=0.2, subsample=0.6, colsample_bytree=1.0,
                        objective='binary:logistic', nthread=4, scale_pos_weight=1, seed=1)


xgb.fit(X_train, X_test, y_train, y_test, cv_folds=5, early_stopping_rounds=50)

TypeError: fit() got an unexpected keyword argument 'cv_folds'

In [ ]:
# dtrain = xgboost.DMatrix(X_train, label=y_train)
# dtest = xgboost.DMatrix(X_test, label=y_test)



# xgb = xgboost.Classifier()

In [81]:
train_data = lightgbm.Dataset(X_train, label=y_train)
test_data = lightgbm.Dataset(X_test, label=y_test)
#
# Train the model
#

parameters = {'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)



[1]	valid_0's auc: 0.792811
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.843068
[3]	valid_0's auc: 0.881484
[4]	valid_0's auc: 0.891012
[5]	valid_0's auc: 0.892135
[6]	valid_0's auc: 0.892899
[7]	valid_0's auc: 0.911829
[8]	valid_0's auc: 0.909334
[9]	valid_0's auc: 0.9148
[10]	valid_0's auc: 0.919808
[11]	valid_0's auc: 0.92242
[12]	valid_0's auc: 0.933301
[13]	valid_0's auc: 0.935832
[14]	valid_0's auc: 0.937008
[15]	valid_0's auc: 0.938743
[16]	valid_0's auc: 0.937291
[17]	valid_0's auc: 0.938098
[18]	valid_0's auc: 0.941946
[19]	valid_0's auc: 0.9386
[20]	valid_0's auc: 0.944658
[21]	valid_0's auc: 0.945482
[22]	valid_0's auc: 0.947659
[23]	valid_0's auc: 0.947861
[24]	valid_0's auc: 0.949444
[25]	valid_0's auc: 0.951173
[26]	valid_0's auc: 0.952406
[27]	valid_0's auc: 0.953318
[28]	valid_0's auc: 0.95304
[29]	valid_0's auc: 0.953983
[30]	valid_0's auc: 0.953544
[31]	valid_0's auc: 0.953277
[32]	valid_0's auc: 0.955915
[33]	valid_0's auc: 0.957

[287]	valid_0's auc: 0.984831
[288]	valid_0's auc: 0.984834
[289]	valid_0's auc: 0.984922
[290]	valid_0's auc: 0.984824
[291]	valid_0's auc: 0.984949
[292]	valid_0's auc: 0.985066
[293]	valid_0's auc: 0.985126
[294]	valid_0's auc: 0.985101
[295]	valid_0's auc: 0.98507
[296]	valid_0's auc: 0.985124
[297]	valid_0's auc: 0.985155
[298]	valid_0's auc: 0.985196
[299]	valid_0's auc: 0.985195
[300]	valid_0's auc: 0.985143
[301]	valid_0's auc: 0.98523
[302]	valid_0's auc: 0.985228
[303]	valid_0's auc: 0.985257
[304]	valid_0's auc: 0.98527
[305]	valid_0's auc: 0.985164
[306]	valid_0's auc: 0.985127
[307]	valid_0's auc: 0.985207
[308]	valid_0's auc: 0.985206
[309]	valid_0's auc: 0.985187
[310]	valid_0's auc: 0.985298
[311]	valid_0's auc: 0.985195
[312]	valid_0's auc: 0.985215
[313]	valid_0's auc: 0.985308
[314]	valid_0's auc: 0.985433
[315]	valid_0's auc: 0.985391
[316]	valid_0's auc: 0.985513
[317]	valid_0's auc: 0.985543
[318]	valid_0's auc: 0.985521
[319]	valid_0's auc: 0.985512
[320]	valid_0